In [1]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


# Overview

This notebook uses my trained pytorch model for vegetation height prediction and runs inference on imagery stack I've prepared.

In [2]:
import torch
from sdm.models import S2DatasetInference
device = torch.device("mps")

# Load the model
model = torch.load("models/veg-height-cnn.pth")
model.to(device)

VegHeightCNN(
  (conv1): Conv2d(7, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0.5, inplace=False)
  (conv3): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (match_dim): Conv2d(7, 1, kernel_size=(1, 1), stride=(1, 1))
  (gate): Sequential(
    (0): Conv2d(7, 1, kernel_size=(1, 1), stride=(1, 1))
    (1): Sigmoid()
  )
)

In [4]:
# Define the dataset and dataloader
from torch.utils.data import DataLoader
from sdm.config import tile_size
dataset = S2DatasetInference(path="data/processed/s2-lidar/s2-lidar-stack.tif", image_size=tile_size)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=12)


/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
# Run inference
from tqdm import tqdm
results = []

model.eval()
with torch.no_grad():
    for inputs, na_mask, spatial_info in tqdm(dataloader):
        
        inputs = inputs.to(device)
        output = model(inputs)
        for idx in range(output.size(0)):            
            results.append({
                "predictions": output[idx].cpu().numpy(),
                "na_mask": na_mask[idx],
                "spatial_info": spatial_info[idx]
            })
        

  0%|          | 0/6 [01:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_